# Train a Hypergraph Neural Network

In this notebook, we will create and train a two-step message passing network in the hypergraph domain. We will use a benchmark dataset, MUTAG (from the TUDataset), to train the model to perform binary classification at the level of the h. 

In [1]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from toponetx import SimplicialComplex

In [3]:
import toponetx.datasets as datasets

In [4]:
from topomodelx.nn.hypergraph.template_layer import TemplateLayer

If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Pre-processing

## Import data ##

The first step is to import the dataset, shrec 16, a benchmark dataset for 3D mesh classification. We then lift each graph into our domain of choice, a hypergraph.

We will also retrieve:
- input signal on the edges for each of these hypergraphs, as that will be what we feed the model in input
- the label associated to the hypergraph

In [50]:
shrec, _ = datasets.mesh.shrec_16(size="small")

# Convert NpzFile objects to NumPy arrays
shrec = {key: np.array(value) for key, value in shrec_training.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
print(x_0s[0].shape)
print(x_0s[1].shape)
print(x_0s[2].shape)
print(x_1s[0].shape)
print(x_1s[1].shape)
print(x_1s[2].shape)

ys = shrec["label"]
simplexes = shrec["complexes"]

hg_list = []
hg_list_test = []
for simplex in simplexes:
    hg = simplex.to_hypergraph()
    hg_list.append(hg)

Loading dataset...

done!
(252, 6)
(252, 6)
(252, 6)
(750, 10)
(750, 10)
(750, 10)


## Define neighborhood structures. ##

Now we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on each simplicial complex. In the case of this architecture, we need the boundary matrix (or incidence matrix) $B_1$ with shape $n_\text{nodes} \times n_\text{edges}$.

In [51]:
incidence_1_list = []
for hg in hg_list:
    incidence_1 = hg.incidence_matrix()
    print(incidence_1.shape)
    incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
    incidence_1_list.append(incidence_1)

(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(250, 1250)
(252, 1250)
(252, 1250)
(250, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(250, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252, 1250)
(252

# Create the Neural Network

Using the TemplateLayer class, we create a neural network with stacked layers.

In [33]:
channels_edge = x_1s[0].shape[1]
channels_node = x_0s[0].shape[1]

In [34]:
class TemplateNN(torch.nn.Module):
    """Neural network implementation of Template for hypergraph classification.

    Parameters
    ---------
    channels_edge : int
        Dimension of edge features
    channels_node : int
        Dimension of node features
    n_layer : 2
        Amount of message passing layers.

    """

    def __init__(self, channels_edge, channels_node, n_layers=2):
        super().__init__()
        layers = []
        for _ in range(n_layers):
            layers.append(
                TemplateLayer(
                    in_channels=channels_edge,
                    intermediate_channels=channels_node,
                    out_channels=channels_edge,
                )
            )
        self.layers = torch.nn.ModuleList(layers)
        self.linear = torch.nn.Linear(channels_edge, 1)

    def forward(self, x_1, incidence_1):
        """Forward computation through layers, then linear layer, then global max pooling.

        Parameters
        ---------
        x_1 : tensor
            shape = [n_edges, channels_edge]
            Edge features.

        incidence_1 : tensor
            shape = [n_nodes, n_edges]
            Boundary matrix of rank 1.

        Returns
        --------
        _ : tensor
            shape = [1]
            Label assigned to whole complex.
        """
        for layer in self.layers:
            x_1 = layer(x_1, incidence_1)
        pooled_x = torch.max(x_1, dim=0)[0]
        return torch.sigmoid(self.linear(pooled_x))[0]

# Train the Neural Network

We specify the model, the loss, and an optimizer.

In [35]:
model = TemplateNN(channels_edge, channels_node, n_layers=2)
model = model.to(device)
crit = torch.nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.1)

Split the dataset into train and test sets.

In [36]:
test_size = 0.2
x_1_train, x_1_test = train_test_split(x_1s, test_size=test_size, shuffle=False)
incidence_1_train, incidence_1_test = train_test_split(
    incidence_1_list, test_size=test_size, shuffle=False
)
y_train, y_test = train_test_split(ys, test_size=test_size, shuffle=False)

The following cell performs the training, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

In [48]:
test_interval = 2
num_epochs = 5
threshold_probability_positive_class = 0.5
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    num_samples = 0
    correct = 0
    model.train()
    for x_1, incidence_1, y in zip(x_1_train, incidence_1_train, y_train):
        x_1 = torch.tensor(x_1)
        print(incidence_1.shape)
        print(x_1.shape)
        x_1, incidence_1, y = (
            x_1.float().to(device),
            incidence_1.float().to(device),
            torch.tensor(y, dtype=torch.float).to(device),
        )
        opt.zero_grad()
        y_hat = model(x_1, incidence_1)
        loss = crit(y_hat, y)
        correct += (
            ((y_hat > threshold_probability_positive_class) == y.bool()).sum().item()
        )
        num_samples += 1
        loss.backward()
        opt.step()
        epoch_loss.append(loss.item())
    train_acc = correct / num_samples
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {train_acc:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            num_samples = 0
            correct = 0
            for x_1, incidence_1, y in zip(x_1_test, incidence_1_test, y_test):
                x_1 = torch.tensor(x_1)
                x_1, incidence_1, y = (
                    x_1.float().to(device),
                    incidence_1.float().to(device),
                    torch.tensor(y, dtype=torch.float).to(device),
                )
                y_hat = model(x_1, incidence_1)
                correct += (
                    ((y_hat > threshold_probability_positive_class) == y.bool())
                    .sum()
                    .item()
                )
                num_samples += 1
            test_acc = correct / num_samples
            print(f"Test_acc: {test_acc:.4f}", flush=True)

torch.Size([252, 1250])
torch.Size([750, 10])


ValueError: Shape of input face features does not have the correct number of edges 1250.